<a href="https://colab.research.google.com/github/rudyselman/trade/blob/master/Swing_Finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

potential trade triggers:
1. low vol pin bar http://prntscr.com/rg3go1
2. Scenario B: consecutive bars of same direction with decreasing vol. http://prntscr.com/rg3kf1
3. if I identify lots of similar swing prices in a short span of time I can do trades to catch stop hunts http://prntscr.com/rgz9js - or swing size to fractal size is high and daily range is small (which would mean theres lots of volatility going nowhere)

Zones creation:

2. http://prntscr.com/rg3kf1 ( Scenario A: high volume bars that are follwed by a lower vol bar and an impulse. Entry could be a low vol opposing candle in zone.  All candles from Scenario B could also be used as a zone. Low vol rejections.  Prior swings at the same price area could be a rating factor.


In [1]:
import pandas as pd
import datetime as dt
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

pd.set_option('display.max_rows', 100)

df = pd.read_csv('drive/My Drive/Colab Notebooks/EURUSD15.csv')

# daily = pd.read_csv('drive/My Drive/Colab Notebooks/EURUSD1440.csv')
# daily = daily.set_axis(['Date', 'Time','O','H','L','C','V'], axis=1, inplace=False)



# # this stuff was to format the original file but is no longer needed

# df = df.set_axis(['Date', 'Time','O','H','L','C','V'], axis=1, inplace=False)
df['DT'] = df['Date'] + ' '+df['Time']
df['DT'] = pd.to_datetime(df['DT'])
df.drop(columns=['Date','Time'],inplace=True)
df = df.reindex(columns=['DT','O','H','L','C','V','D_Range','ADR','Frac_H','Frac_L'])
# df['Datetime'] -= pd.Timedelta(hours=7) # minus 7 hours to match CST



# if you want to re-iterate over the highs and lows un-comment these lines:

df['Locked_H'] = 0
df['Locked_L'] = 0
df['Swing'] = 0

df = df[64000:]

df.reset_index(drop=True,inplace=True)  # reset the index (numbers are missing)

df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,DT,O,H,L,C,V,D_Range,ADR,Frac_H,Frac_L,Locked_H,Locked_L,Swing
0,2020-02-25 14:15:00,1.08358,1.08375,1.08341,1.08349,998,0.00601,0.004922,NaN,NaN,0,0,0
1,2020-02-25 14:30:00,1.08350,1.08381,1.08342,1.08378,775,0.00601,0.004925,NaN,NaN,0,0,0
2,2020-02-25 14:45:00,1.08377,1.08381,1.08300,1.08361,902,0.00601,0.004928,NaN,NaN,0,0,0
3,2020-02-25 15:00:00,1.08362,1.08406,1.08356,1.08399,876,0.00601,0.004930,NaN,NaN,0,0,0
4,2020-02-25 15:15:00,1.08399,1.08472,1.08392,1.08467,955,0.00601,0.004933,NaN,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1278,2020-03-13 21:45:00,1.10696,1.11147,1.10637,1.11132,2185,0.01664,0.015202,NaN,NaN,0,0,0
1279,2020-03-13 22:00:00,1.11132,1.11168,1.10807,1.10807,1847,0.01664,0.015209,NaN,NaN,0,0,0
1280,2020-03-13 22:15:00,1.10806,1.10975,1.10796,1.10956,1120,0.01664,0.015216,NaN,NaN,0,0,0
1281,2020-03-13 22:30:00,1.10957,1.11157,1.10946,1.11078,1220,0.01664,0.015222,NaN,NaN,0,0,0


# Get Daily Range 



In [0]:

for i in df['DT'].dt.date.unique():
  group = df.groupby(df['DT'].dt.date).get_group(i)
  dayrange = max(group['H']) - min(group['L'])
  df.loc[group.index,'D_Range'] = dayrange


In [3]:
df.tail()

,DT,O,H,L,C,V,D_Range,ADR,Frac_H,Frac_L,Locked_H,Locked_L,Swing
1278,2020-03-13 21:45:00,1.10696,1.11147,1.10637,1.11132,2185,0.01664,0.015202,NaN,NaN,0,0,0
1279,2020-03-13 22:00:00,1.11132,1.11168,1.10807,1.10807,1847,0.01664,0.015209,NaN,NaN,0,0,0
1280,2020-03-13 22:15:00,1.10806,1.10975,1.10796,1.10956,1120,0.01664,0.015216,NaN,NaN,0,0,0
1281,2020-03-13 22:30:00,1.10957,1.11157,1.10946,1.11078,1220,0.01664,0.015222,NaN,NaN,0,0,0
1282,2020-03-13 22:45:00,1.11077,1.11158,1.10967,1.10986,967,0.01664,0.015229,NaN,NaN,0,0,0


alright groupby is my forever friend from now on. 

#Get ADR

There are 96 15m candles per day and I want a 10 day average.  This will give me an index error so it needs a try block

In [0]:
for i in range(0, len(df)):
    try:
      df.loc[i,'ADR'] = round(df.loc[i-960:i,'D_Range'].mean(),5)
    except:
      pass

# Identify Fractals

For each bar, walk forwards and backwards to see how far you get while the high remains highest high. Do this individually for both the highs and the lows. While walking forward see how far price moved away from that fractal. The smaller of the 2 counts will be used as the fractal rating.

There are 4 try blocks total for the highs, lows, forwards, and backwards. Exceptions are hit when the loop reaches either end of the df. I need these to know whether a row can be locked from further iterations.

While a market is unfolding the fractals will need to be recalculated for each bar until the forward walk value has been determined with no errors. Once that happens the value will be locked in place and even though prior fractals may need to be reiterated over, the locked ones will not. This pic show's a locked fractal while a previous fractal needs to still be re-iterated into the future. http://prntscr.com/rhc9zt

Unfortunately to account for edge cases I need to create a column specific to highs and lows. 

There is also a time limit for a fractal to be locked.  Once it is the highest high/lowest low for 120 bars it will be considered a major swing and there's no need to know the exact bar count beyond that.

In [0]:
# %%timeit
highs = df.loc[:,'H'].values
lows = df.loc[:,'L'].values 

for i in range(0, len(df)): 
  low = lows[i]
  high = highs[i]
 
  # the order below is highs forwards, highs backwards

  if not df.loc[i,'Locked_H']:
    try:   # iter forwards 
      count_prior = 0   # initialize counters and such
      count_next = 0
      next_ = i + 1
      prior_ = i - 1

      while high > highs[next_] and count_next < 120: # iter limit
        next_ +=1
        count_next +=1
      df.loc[i,'Locked_H'] = 1 # if it hasn't error'd lock the row 
    except:
      pass

    try: # iter backwards
      while high >= highs[prior_] and count_prior < 120: 
        prior_ -=1
        count_prior +=1
    except:
      pass 
    df.loc[i,'Frac_H'] = min([count_prior, count_next])
    try: # this will return the swing size if frac > 1
      if df.loc[i,'Frac_H'] > 1:  # the 'min is an empty argument' requires try block
        df.loc[i,'Swing'] = min(lows[i+1:i+count_next]) - high
    except:
      df.loc[i,'Swing'] = 0

  # now the same thing but for the lows
  if not df.loc[i,'Locked_L']:
    try: # iter forwards 
      count_prior = 0
      count_next = 0
      next_ = i + 1
      prior_ = i - 1

      while low < lows[next_] and count_next < 120: 
        next_ +=1
        count_next +=1
      df.loc[i,'Locked_L'] = 1 # if it hasn't error'd set the lock value 
    except:
      pass

    try: # iter backwards
      while low <= lows[prior_] and count_prior < 120: 
        prior_ -=1
        count_prior +=1
      
    except:
      pass
    df.loc[i,'Frac_L'] = min([count_prior, count_next])

    # find swing distance
    # only overwrite if the new value is greater than the last
    try:  
      if (max(highs[i:i+count_next]) - low > abs(df.loc[i,'Swing'])
          and df.loc[i,'Frac_L'] > 1):
        df.loc[i,'Swing'] = max(highs[i+1:i+count_next]) - low
    except:
        pass


In [6]:
df.head(30)

,DT,O,H,L,C,V,D_Range,ADR,Frac_H,Frac_L,Locked_H,Locked_L,Swing
0,2020-02-25 14:15:00,1.08358,1.08375,1.08341,1.08349,998,0.00601,0.00601,0.0,1.0,1,1,0.00000
1,2020-02-25 14:30:00,1.08350,1.08381,1.08342,1.08378,775,0.00601,0.00601,0.0,0.0,1,1,0.00000
2,2020-02-25 14:45:00,1.08377,1.08381,1.08300,1.08361,902,0.00601,0.00601,0.0,120.0,1,1,0.00786
3,2020-02-25 15:00:00,1.08362,1.08406,1.08356,1.08399,876,0.00601,0.00601,0.0,0.0,1,1,0.00000
4,2020-02-25 15:15:00,1.08399,1.08472,1.08392,1.08467,955,0.00601,0.00601,0.0,0.0,1,1,0.00000
5,2020-02-25 15:30:00,1.08465,1.08496,1.08424,1.08442,1092,0.00601,0.00601,5.0,0.0,1,1,-0.00178
6,2020-02-25 15:45:00,1.08445,1.08447,1.08390,1.08408,830,0.00601,0.00601,0.0,0.0,1,1,0.00000
7,2020-02-25 16:00:00,1.08408,1.08456,1.08345,1.08416,1066,0.00601,0.00601,1.0,2.0,1,1,0.00092
8,2020-02-25 16:15:00,1.08417,1.08437,1.08387,1.08426,984,0.00601,0.00601,0.0,0.0,1,1,0.00000
9,2020-02-25 16:30:00,1.08425,1.08457,1.08390,1.08437,1584,0.00601,0.00601,2.0,0.0,1,1,-0.00139


# Swings

I need to know the swing size in relation to the average daily range in order for it to mean anything.

Also, a fractal size needs to correlate to its swing size.  A large fractal with a tiny swing is mostly irrelevent as that would signify a sideways market. Likewise a large swing on a tiny fractal would signify it was just a slight pullback within a larger swing. Here is an example: http://prntscr.com/rhc2qh

In that situation it would be best to only look at the fractals from further up. 

So I also need a swing rating which will be a combination of fractal size and swing%.

In [0]:
# get the swing in relation to ADR
df['Swing%'] = round(df['Swing']/df['ADR'], 2)

# get the swing rating in relation to fractal size
temp = df[abs(df['Swing']) > 0]
df['Sw_Rating'] = (abs(temp['Swing%']*15) + temp['Frac_H'] + temp['Frac_L']) / 2
# technically thats not perfect but it will work for most cases
# and I can come back to it later


```
df['Sw_Rating'].describe()

count    17555.000000
mean         8.498616
std         12.295569
min          1.075000
25%          2.425000
50%          4.150000
75%          8.700000
max         94.975000
Name: Sw_Rating, dtype: float64
```
# Conclusion

My original idea was just to map peaks and troughs so I can identify when certain patterns happen.  I can use the swing ratings though for a variety of things like:
- creating adaptive support and resistance zones that will remain valid for a time period in relation to the swing ratings that created it
- looking for small swings near each other where each one is progressively higher or lower and volume is fading
- other cool things I can't think of

In [0]:

# overwrite the file

# df.to_csv('EURUSD15test.csv', index=False)
# !cp EURUSD15test.csv "drive/My Drive/Colab Notebooks/"  
 

In [0]:
# xx.sort_index()

In [12]:
import plotly.graph_objects as go
df = df[50:]

fig = go.Figure(data=[go.Candlestick(x=df['DT'],
                open=df['O'], high=df['H'],
                low=df['L'], close=df['C'])
                      ])
xx = df['DT'][df['Swing'] < 0].append(df['DT'][df['Swing'] > 0])
xx = xx.sort_index()
yy = df['H'][df['Swing'] < 0].append(df['L'][df['Swing'] > 0])
yy = yy.sort_index()
# yy = df['L'][df['Sw_Rating'] > 0]
aa = df
fig.add_trace(go.Scatter(
    x=xx,
    y=yy,
    mode="lines+markers+text",
    name="Lines, Markers and Text",
    # text=str(yy.values),
    textposition="top center"
))
fig.show()

# len(df['DT'][df['Sw_Rating'] > 0])
len(xx)

36